## Starbucks 서울 매장 위치 Crawling

## 사용 모듈 
- Selenium
- BeautifulSoup
- Pandas
- Forium

In [4]:
%pip install beautifulsoup4

# 글로벌에서만 설치하고 가상환경에서 설치 안해서 재설치 

                                              0.0/143.0 kB ? eta -:--:--
     -------------------------------------- 143.0/143.0 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By  # html 파싱용
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'   
driver.get(url)

In [20]:
btn_seoul_link = '# container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(4) > a'

driver.find_element(By.CSS_SELECTOR, btn_seoul_link).click()

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=116.0.5845.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6798F52A2+57122]
	(No symbol) [0x00007FF67986EA92]
	(No symbol) [0x00007FF67973E3AB]
	(No symbol) [0x00007FF679742382]
	(No symbol) [0x00007FF679743D2F]
	(No symbol) [0x00007FF679743E20]
	(No symbol) [0x00007FF6797779B1]
	(No symbol) [0x00007FF679777E2C]
	(No symbol) [0x00007FF6797B0B67]
	(No symbol) [0x00007FF67979701F]
	(No symbol) [0x00007FF6797AEB82]
	(No symbol) [0x00007FF679796DB3]
	(No symbol) [0x00007FF67976D2B1]
	(No symbol) [0x00007FF67976E494]
	GetHandleVerifier [0x00007FF679B9EF82+2849794]
	GetHandleVerifier [0x00007FF679BF1D24+3189156]
	GetHandleVerifier [0x00007FF679BEACAF+3160367]
	GetHandleVerifier [0x00007FF679986D06+653702]
	(No symbol) [0x00007FF67

In [21]:
btn_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_all).click()

In [25]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [27]:
len(soup.select('li.quickResultLstCon'))

601

In [32]:
sb_seoul_list = soup.select('li.quickResultLstCon')
sb_seoul_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [51]:
# 매장 정보 
sb_store = sb_seoul_list[0]
name = sb_store.select('strong')[0].text.strip()    # 매장 이름 
lat = item['data-lat']   # 경도 값 
lng = sb_store['data-long']   # 경도 값 
addr = sb_store.select('p')[0].get_text(separator='|').split('|')[0]
number = sb_store.select('p')[0].get_text(separator='|').split('|')[1]
type = sb_store.select('i')[0]['class'][0].split('_')

In [52]:
sb_list = []

for item in sb_seoul_list:
    name = item.select('strong')[0].text.strip()    # 매장 이름 
    lat = item['data-lat']   # 경도 값 
    lng = item['data-long']   # 경도 값 
    addr = item.select('p')[0].get_text(separator='|').split('|')[0]
    number = item.select('p')[0].get_text(separator='|').split('|')[1]
    type = item.select('i')[0]['class'][0].split('_')

    sb_list.append([name, lat, lng, addr, number, type])

#### tqdm

프로그래스바로 진행사항을 표시해주는 모듈

```
!pip install tqdm
```

In [45]:
%pip install tqdm

                                              0.0/78.3 kB ? eta -:--:--
     ---------------------------------------- 78.3/78.3 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
from tqdm.autonotebook import tqdm

In [53]:
sb_list=[]


for item in sb_seoul_list:
    name = item.select('strong')[0].text.strip()    # 매장 이름 
    lat = item['data-lat']   # 경도 값 
    lng = item['data-long']   # 경도 값 
    addr = item.select('p')[0].get_text(separator='|').split('|')[0]
    number = item.select('p')[0].get_text(separator='|').split('|')[1]
    type = item.select('i')[0]['class'][0].split('_')

    sb_list.append([name, lat, lng, addr, number, type])

In [57]:
len(sb_list)

601

In [59]:
# dataframe에 저장
colunms =['매장명', '위도', '경도', '주소', '전화번호', '매장타입']
df_sb_seoul = pd.DataFrame(data=sb_list, columns=colunms)
df_sb_seoul.head()

,매장명,위도,경도,주소,전화번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,"[pin, general]"
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,"[pin, general]"
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,"[pin, general]"
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,"[pin, general]"
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,"[pin, reserve]"


In [60]:
df_sb_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     601 non-null    object
 1   위도      601 non-null    object
 2   경도      601 non-null    object
 3   주소      601 non-null    object
 4   전화번호    601 non-null    object
 5   매장타입    601 non-null    object
dtypes: object(6)
memory usage: 28.3+ KB


In [61]:
df_sb_seoul.to_excel('./starbucks_seoul.xlsx', index=False)

#### 시각화 , 전처리 

- 지도 모듈 설치 
    - folium

- !pip install folium

In [62]:
%pip install folium

  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
                                              0.0/62.6 kB ? eta -:--:--
     ---------------------------------------- 62.6/62.6 kB ? eta 0:00:00
                                              0.0/96.6 kB ? eta -:--:--
     ---------------------------------------- 96.6/96.6 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
df_sb_seoul

,매장명,위도,경도,주소,전화번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,"[pin, general]"
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,"[pin, general]"
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,"[pin, general]"
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,"[pin, general]"
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,"[pin, reserve]"
...,...,...,...,...,...,...
596,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,"[pin, general]"
597,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,"[pin, general]"
598,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,"[pin, general]"
599,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,"[pin, general]"


In [65]:
df_sb_seoul['주소'][600].split()[1]

'중랑구'

In [66]:
gu_list = []

for item in df_sb_seoul['주소']:
    gu = item.split()[1]
    gu_list.append(gu)


In [69]:
df_sb_seoul['구'] = gu_list

In [70]:
df_sb_seoul

,매장명,위도,경도,주소,전화번호,매장타입,구
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,"[pin, general]",강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,"[pin, general]",강남구
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,"[pin, general]",강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,"[pin, general]",강남구
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,"[pin, reserve]",강남구
...,...,...,...,...,...,...,...
596,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,"[pin, general]",중랑구
597,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,"[pin, general]",중랑구
598,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,"[pin, general]",중랑구
599,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,"[pin, general]",중랑구


#### 대한민국 행정구역 위경도 전처리 

In [71]:
df_loc = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
df_loc

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275
...,...,...,...,...,...
290,충청충주시,충청,충주시,127.928144,36.988181
291,충청태안군,충청,태안군,126.299975,36.742667
292,충청한누리대로,충청,한누리대로,127.289926,36.485450
293,충청홍성군,충청,홍성군,126.662908,36.598361


In [74]:
df_loc_seoul = df_loc[df_loc['do']=='서울']
df_loc_seoul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 138 to 162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     25 non-null     object 
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.2+ KB


In [76]:
df_gu_count=df_sb_seoul.pivot_table(index='구', values='매장명', aggfunc='count').rename(columns={'매장명':'스타벅스 매장수'})
df_gu_count

,스타벅스 매장수
구,
강남구,88
강동구,17
강북구,6
강서구,25
관악구,12
광진구,18
구로구,14
금천구,12
노원구,14


In [77]:
df_loc_seoul.rename(columns={'do': '도', 'city' : '구', 'latitude':'위도', 'longitude':'경도'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_18640\2252605461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc_seoul.rename(columns={'do': '도', 'city' : '구', 'latitude':'위도', 'longitude':'경도'}, inplace=True)


In [78]:
df_loc_seoul.reset_index(drop=True, inplace=True)
df_loc_seoul.tail()

,docity,도,구,경도,위도
20,서울용산구,서울,용산구,126.967522,37.536094
21,서울은평구,서울,은평구,126.931242,37.599969
22,서울종로구,서울,종로구,126.981642,37.570378
23,서울중구,서울,중구,126.999642,37.561003
24,서울중랑구,서울,중랑구,127.094778,37.603806


In [79]:
df_loc_seoul[['도','구','위도','경도']]
df_loc_seoul

,docity,도,구,경도,위도
0,서울강남구,서울,강남구,127.049556,37.514575
1,서울강동구,서울,강동구,127.125864,37.527367
2,서울강북구,서울,강북구,127.027719,37.636956
3,서울강서구,서울,강서구,126.851675,37.548156
4,서울관악구,서울,관악구,126.953844,37.475386
5,서울광진구,서울,광진구,127.084533,37.535739
6,서울구로구,서울,구로구,126.889597,37.492650
7,서울금천구,서울,금천구,126.904197,37.449108
8,서울노원구,서울,노원구,127.058389,37.651461
9,서울도봉구,서울,도봉구,127.049522,37.665833


In [81]:
df_sb_seoul_stat= pd.merge(df_loc_seoul, df_gu_count, how='left', on='구')
df_sb_seoul_stat.head()

,docity,도,구,경도,위도,스타벅스 매장수
0,서울강남구,서울,강남구,127.049556,37.514575,88
1,서울강동구,서울,강동구,127.125864,37.527367,17
2,서울강북구,서울,강북구,127.027719,37.636956,6
3,서울강서구,서울,강서구,126.851675,37.548156,25
4,서울관악구,서울,관악구,126.953844,37.475386,12


In [82]:
import folium

In [85]:
sb_map = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)

for idx in df_sb_seoul.index:
    lat = df_sb_seoul.loc[idx, '위도']
    lng = df_sb_seoul.loc[idx, '경도']
    name = df_sb_seoul.loc[idx, '매장명']

    folium.CircleMarker(location=[lat, lng], popup=name).add_to(sb_map)

sb_map